<div class='alert alert-block alert-warning' style="font-size: 20pt; font-family: cursive; color: #08415C"> 
    <b> <center> Web Scraping Data</center> </b>
</div>

<div style="font-size: 13pt; font-family: verdana; color: #08415C"> 
    The dataset has a total of 6 columns.
    <ul>
        <li>name - name of the residential complex</li>
        <li>link - link from the site</li>
        <li>address - address of the residential complex</li>    
        <li>square_price - price per square meter</li>
        <li>Area - the area of the apartment</li>
        <li>price_start_from - inital prices for an apartment</li>
    </ul>
</div>

In [12]:
import requests
from bs4 import BeautifulSoup
from time import sleep

In [13]:
url = "https://krisha.kz/complex/search/"

r = requests.get(url)

In [14]:
soup = BeautifulSoup(r.text, 'html')

In [15]:
soup

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>Новостройки Казахстан: 🏘 новые квартиры в ЖК от застройщика - Крыша</title>
<meta content="Все ⭐ новостройки и жилые комплексы ⭐ в Казахстане. Планировки, цены, описание, расположение на карте и контакты всех ЖК и новостроек на сайте Крыша." name="description"/>
<link href="https://krisha.kz/complex/search/" hreflang="ru" rel="alternate"/>
<link href="https://krisha.kz/kz/complex/search/" hreflang="kz" rel="alternate"/>
<link href="https://krisha.kz/complex/search/" rel="canonical"/>
<link href="https://m.krisha.kz/complex/search/" media="only screen and (max-width: 640px)" rel="alternate"/>
<link href="//krisha.kz/static/frontend/favicons/favicon.ico?v2" rel="icon" type="image/x-icon"/>
<link href="//krisha.kz/static/frontend/favicons/favicon-64x64.png?v2" rel="icon" sizes="64x64" type="image/png"/>
<l

In [16]:
main_card = soup.find('div', class_='complex-card complex-card--not-empty complex-card--with-result')
main_card

<div class="complex-card complex-card--not-empty complex-card--with-result" data-complex-alias="almaty/ramscity" data-complex-id="11653762" data-complex-region="Алматы" data-filter-city="2" data-free-call-visibility="" data-has-buttons="1" data-is-free="" data-name="RAMS CITY" data-work-time-from="09:00" data-work-time-to="18:00">
<div class="complex-card__state">
                Строящийся
            </div>
<a class="complex-card__main-block complex-card__main-block--hypothec" href="/complex/show/almaty/ramscity/" target="_blank">
<div class="complex-card__child-block flats">
<div class="complex-card__info">
<div class="complex-card__header"><picture>
<img alt="RAMS CITY" loading="lazy" src="https://photos-kr.kcdn.kz/00/e1ff3fd6b9aaa7b1eb7ea1b71edc37fec75425/photo-448x280.jpg"/>
</picture>
</div>
</div>
<div class="complex-card__content">
<span class="credit-badge credit-badge--hypothec-full">
<i class="credit-badge__icon fi-home-bordered"></i>
                                    Ипо

In [17]:
link = 'https://krisha.kz' + main_card.find('a', class_="complex-card__main-block complex-card__main-block--hypothec").get("href")
link

'https://krisha.kz/complex/show/almaty/ramscity/'

In [18]:
name = main_card.find('a', class_="complex-card__main-block complex-card__main-block--hypothec").find('p', class_='complex-card__title').text.strip()
name

'ЖК RAMS CITY'

In [19]:
address = main_card.find('a', class_="complex-card__main-block complex-card__main-block--hypothec").find('p', class_='complex-card__address').text.strip()
address

'Алматы, Бостандыкский р-н, ул. Жандосова, 94А'

In [20]:
square_price = ''.join(filter(str.isdigit, main_card.find('a', class_="complex-card__main-block complex-card__main-block--hypothec").find('p', class_='complex-card__square-price').text)) + 'tg'
square_price

'590000²tg'

In [21]:
Area = main_card.find('a', class_="complex-card__main-block complex-card__main-block--hypothec").find('div', class_='complex-card__price-item complex-card__price-item--result').find('span').text
Area

'2-комн. 51.4 м²'

In [22]:
price_start_from = ''.join(filter(str.isdigit, main_card.find('a', class_="complex-card__main-block complex-card__main-block--hypothec").find('span', class_='complex-flat-price').text))
price_start_from

'38373441'

In [23]:
text = price_start_from
numeric_value = ''.join(filter(str.isdigit, text))
price_start_from = int(numeric_value)
price_start_from

38373441

In [24]:
flats = soup.findAll('div', class_='complex-card complex-card--not-empty complex-card--with-result')
len(flats)

11

In [25]:
flat_dict = {}

for p in range(1, 88):
    
    print('Web scraping page:', p)
    
    url = f"https://krisha.kz/complex/search/ramscity/?state[]=1&state[]=2&page={p}"
    r = requests.get(url)
    sleep(2)
    soup = BeautifulSoup(r.text, 'lxml')
    
    flats = soup.findAll('a', class_="complex-card__main-block complex-card__main-block--hypothec")
    
    

    for flat in flats:
        
        flat_name = flat.find('p', class_='complex-card__title').text.strip()
        flat_dict[flat_name] = {}  # Create a nested dictionary for each flat
        
        try:
            link = 'https://krisha.kz' + str(flat.get("href"))
            flat_dict[flat_name]['link'] = link
        except AttributeError:
            flat_dict[flat_name]['link'] = None
            
        try:
            address_element = flat.find('p', class_='complex-card__address')
            flat_dict[flat_name]['address'] = address_element.text.strip()
        except AttributeError:
            flat_dict[flat_name]['address'] = None

        try:
            square_price_element = flat.find('p', class_='complex-card__square-price')
            square_price = ''.join(filter(str.isdigit, square_price_element.text)) + 'tg'
            flat_dict[flat_name]['square_price'] = square_price
        except AttributeError:
            flat_dict[flat_name]['square_price'] = None

        try:
            area_element = flat.find('div', class_='complex-card__price-item complex-card__price-item--result').find('span')
            flat_dict[flat_name]['Area'] = area_element.text
        except AttributeError:
            flat_dict[flat_name]['Area'] = None

        try:
            price_element = flat.find('span', class_='complex-flat-price')
            price_start_from = ''.join(filter(str.isdigit, price_element.text))
            flat_dict[flat_name]['price_start_from'] = price_start_from
        except AttributeError:
            flat_dict[flat_name]['price_start_from'] = None

    

# Now, the 'data' dictionary contains data for each flat with possible missing values set to None


Web scraping page: 1
Web scraping page: 2
Web scraping page: 3
Web scraping page: 4
Web scraping page: 5
Web scraping page: 6
Web scraping page: 7
Web scraping page: 8
Web scraping page: 9
Web scraping page: 10
Web scraping page: 11
Web scraping page: 12
Web scraping page: 13
Web scraping page: 14
Web scraping page: 15
Web scraping page: 16
Web scraping page: 17
Web scraping page: 18
Web scraping page: 19
Web scraping page: 20
Web scraping page: 21
Web scraping page: 22
Web scraping page: 23
Web scraping page: 24
Web scraping page: 25
Web scraping page: 26
Web scraping page: 27
Web scraping page: 28
Web scraping page: 29
Web scraping page: 30
Web scraping page: 31
Web scraping page: 32
Web scraping page: 33
Web scraping page: 34
Web scraping page: 35
Web scraping page: 36
Web scraping page: 37
Web scraping page: 38
Web scraping page: 39
Web scraping page: 40
Web scraping page: 41
Web scraping page: 42
Web scraping page: 43
Web scraping page: 44
Web scraping page: 45
Web scraping page: 

In [26]:
len(flat_dict)

795

In [36]:
import pandas as pd

df = pd.DataFrame(flat_dict)

df = df.transpose()

pd.set_option('display.max.rows', 834)

df.head()

,link,address,square_price,Area,price_start_from
ЖК RAMS EVO,https://krisha.kz/complex/show/almaty/ramsevo/,"Алматы, Алмалинский р-н, пересечение Ауэзова и...",tg,None,None
ЖК RAMS CITY,https://krisha.kz/complex/show/almaty/ramscity/,"Алматы, Бостандыкский р-н, ул. Жандосова, 94А",590000²tg,2-комн. 51.4 м²,38373441
ЖК RAMS SAIAHAT,https://krisha.kz/complex/show/almaty/ramssaia...,"Алматы, Жетысуский р-н, пересечении улиц Райым...",595000²tg,1-комн. 37.6 м²,25229600
ЖК Столичный 2,https://krisha.kz/complex/show/nur-sultan/stol...,"Астана, Сарыарка р-н, ул. Абая, 10/2",400000²tg,1-комн. 39.4 м²,16548000
ЖК Qarasai Park,https://krisha.kz/complex/show/almaty/qarasaip...,"Алматы, Алмалинский р-н, ул. Карасай батыра, 1...",645000²tg,1-комн. 59.2 м²,56240000


In [37]:
import re 

df['square_price'] = df['square_price'].astype(str)
# Use regular expressions to extract the numeric part
df['square_price'] = df['square_price'].str.extract(r'(\d+)')

new_order = ['address', 'Area', 'square_price', 'price_start_from', 'link']

df = df[new_order]

df.head()

,address,Area,square_price,price_start_from,link
ЖК RAMS EVO,"Алматы, Алмалинский р-н, пересечение Ауэзова и...",None,NaN,None,https://krisha.kz/complex/show/almaty/ramsevo/
ЖК RAMS CITY,"Алматы, Бостандыкский р-н, ул. Жандосова, 94А",2-комн. 51.4 м²,590000,38373441,https://krisha.kz/complex/show/almaty/ramscity/
ЖК RAMS SAIAHAT,"Алматы, Жетысуский р-н, пересечении улиц Райым...",1-комн. 37.6 м²,595000,25229600,https://krisha.kz/complex/show/almaty/ramssaia...
ЖК Столичный 2,"Астана, Сарыарка р-н, ул. Абая, 10/2",1-комн. 39.4 м²,400000,16548000,https://krisha.kz/complex/show/nur-sultan/stol...
ЖК Qarasai Park,"Алматы, Алмалинский р-н, ул. Карасай батыра, 1...",1-комн. 59.2 м²,645000,56240000,https://krisha.kz/complex/show/almaty/qarasaip...


In [39]:
# Save to CSV
df.to_csv('krishaKZ.csv', index=False)  # Set index=False to exclude the DataFrame index in the CSV file